#**<center>LAB-03**

###**Student id:** 202211037

###**Student Name:** Aksh Patel

In [ ]:
!pip install --quiet mrjob==0.7.4

In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Exercise 1

In [ ]:
%%file broadcastJoin.py
from mrjob.job import MRJob

class Broadjoin(MRJob):
  def mapper_init(self):
    f = open("/content/gdrive/MyDrive/big_data_processing/datasets/mr/empc.csv").read()
    self.empc = f.split("\n")
    self.list1 = []

    for i in self.empc:
      self.list1.append(list(map(str,i.split(","))))

    self.map1 = {}

    for i in range(len(self.list1)):
      if self.list1[i] != [""]:
        if self.list1[i][6] in self.map1:
          self.map1[self.list1[i][6]].append(self.list1[i])
        else:
          self.map1[self.list1[i][6]] = [self.list1[i]]

  def mapper(self, key, line):
    record = line.split(",")
    dno = record[0]

    if dno in self.map1.keys():
      for values in self.map1[dno]:
        yield (record,values)

if __name__ == '__main__':
  Broadjoin.run()

Overwriting broadcastJoin.py


In [ ]:
!python broadcastJoin.py "/content/gdrive/My Drive/big_data_processing/datasets/mr/depc.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/broadcastJoin.root.20230826.174032.852983
Running step 1 of 1...
job output is in /tmp/broadcastJoin.root.20230826.174032.852983/output
Streaming final output from /tmp/broadcastJoin.root.20230826.174032.852983/output...
["4", "Administration", "106", "1985-01-01"]	["106", "Jennifer", "1931-06-20", "F", "43000", "105", "4"]
["4", "Administration", "106", "1985-01-01"]	["107", "Ahmad", "1959-03-29", "M", "25000", "106", "4"]
["4", "Administration", "106", "1985-01-01"]	["108", "Alicia", "1958-07-19", "F", "25000", "106", "4"]
["1", "Headquater", "105", "1971-06-19"]	["105", "James", "2027-11-10", "M", "55000", "", "1"]
["5", "Research", "102", "1978-05-22"]	["101", "John", "1955-01-09", "M", "30000", "102", "5"]
["5", "Research", "102", "1978-05-22"]	["102", "Franklin", "1945-12-08", "M", "40000", "105", "5"]
["5", "Research", "102", "1978-05-22"]	["103", "Joyce", "19

## Exercise 2

In [ ]:
%%file MovWindow.py
from mrjob.job import MRJob
import re
import shlex


class MovWindow(MRJob):

    def reducer_init(self):

        self.n = 31
        self.iter = 0
        self.arr = []
        self.val1 = 0
        self.val2 = 0

    def mapper(self, key, line):

        if "Row" not in line:

            record = shlex.shlex(line, posix=True)
            record.whitespace += ','
            record.whitespace_split = True

            record = list(record)
            id = int(record[0])
            date = record[3]

            firstTime = int(record[6].replace(',', ''))
            repeatTime = int(record[7].replace(',', ''))
            yield None, (id,date, firstTime, repeatTime)

    def reducer(self, key, values):

        for i in list(values):
            if self.iter < self.n:
                self.val1 += i[2]
                self.val2 += i[3]
                self.iter += 1
            elif self.iter == self.n:
                self.val1 += i[2] - self.arr[0][0]
                self.val2 += i[3] - self.arr[0][1]
                del self.arr[0]
            self.arr.append([i[2],i[3]])
            yield i[0], (i[1], self.val1/self.iter, self.val2/self.iter)


if __name__ == '__main__':
    MovWindow.run()

Writing MovWindow.py


In [ ]:
!python3 MovWindow.py "/content/gdrive/MyDrive/big_data_processing/datasets/mr/daily-website-visitors.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/MovWindow.root.20230826.182335.511534
Running step 1 of 1...
job output is in /tmp/MovWindow.root.20230826.182335.511534/output
Streaming final output from /tmp/MovWindow.root.20230826.182335.511534/output...
1	["9/14/2014", 1430.0, 152.0]
2	["9/15/2014", 1863.5, 191.5]
3	["9/16/2014", 2026.3333333333333, 220.33333333333334]
4	["9/17/2014", 2101.5, 237.0]
5	["9/18/2014", 2107.2, 236.8]
6	["9/19/2014", 2026.3333333333333, 237.5]
7	["9/20/2014", 1877.5714285714287, 222.57142857142858]
8	["9/21/2014", 1828.0, 216.625]
9	["9/22/2014", 1881.7777777777778, 223.0]
10	["9/23/2014", 1992.5, 227.5]
11	["9/24/2014", 2074.181818181818, 232.63636363636363]
12	["9/25/2014", 2129.9166666666665, 237.08333333333334]
13	["9/26/2014", 2122.4615384615386, 235.46153846153845]
14	["9/27/2014", 2045.142857142857, 228.64285714285714]
15	["9/28/2014", 2016.3333333333333, 226.26666666666668]


##**Colab Notebook link:**

[202211037_BDP_lab03.ipynb](https://colab.research.google.com/drive/1fOZMCSSj-kNILqbRnl89rBzz_yGdyczC?usp=chrome_ntp#scrollTo=7WnCBDS5U6e0)